## Genetic algorithm for FLP and LRP

This algorithm aims to utilize hybrid characteristics to solve the Facility location problem and its extention the location routing problem. The routing component is optional in the algorithm.

In [63]:
### Library import

import numpy as np
import pandas as pd
import math
import random

In [64]:
### Data import

filename = "instance287.txt"

with open(filename) as f:
    lines = f.readlines()
    NoCustomers = int(lines[0])
    X = []
    Y = []
    Dem = []
    for i in lines[1:]:
        X.append(float(i.split()[0]))
        Y.append(float(i.split()[1]))
        Dem.append(float(i.split()[2]))

        
Xcord = np.asarray(X).reshape(NoCustomers,1)
Ycord = np.asarray(Y).reshape(NoCustomers,1)
Demand = np.asarray(Dem).reshape(NoCustomers,1)


In [65]:
### Model parameters

NoFacil = 5
NoChromo = 300





In [66]:
np.random.seed(100)
ChromoX = np.random.rand(NoChromo,NoFacil)*100
np.random.seed(120)
ChromoY = np.random.rand(NoChromo,NoFacil)*100

#print(ChromoX)
#print("...")
#print(ChromoY)

In [67]:
def get_Dist(X1,Y1,X2,Y2):
    return math.sqrt(math.pow(X1-X2,2)+math.pow(Y1-Y2,2))

In [68]:
def Facil_distance(Xcord,Ycord,ChromoX,ChromoY,NoCustomers):
    
    d = np.zeros((NoCustomers))
    for j in range(NoCustomers):
        d[j] = get_Dist(ChromoX,ChromoY,Xcord[j],Ycord[j])
    
    return d

In [69]:
def Evaluation(Xcord, Ycord,Dem,ChromoX,ChromoY,NoCustomers,NoFacil,NoChromo):
    
    print(NoChromo)
    result =np.zeros((NoChromo))
    for i in range(NoChromo):
        d = np.zeros((NoCustomers,NoFacil))
        proximity = np.zeros((NoCustomers))
        for j in range(NoFacil):
            print(ChromoY[i,j])
            d[:,j]=Facil_distance(Xcord,Ycord,ChromoX[i,j],ChromoY[i,j],NoCustomers)
    
        result[i]=(np.sum(np.multiply(d.min(axis = 1),Dem.T)))
        proximity=(d.argmin(axis=1))
    
    
    
    return result

In [70]:
def Weiszfeld(Xcord,Ycord,Xcenter,Ycenter,NoCustomers,Dem):
    
    improv = 1 
    epsilon = 10**(-12)
    
    
    Cost = (np.sum(np.multiply(Facil_distance(Xcord,Ycord,Xcenter,Ycenter,NoCustomers),Dem.T)))
    
    while (improv>epsilon):
        
        x = 0
        y = 0
        W = 0
        d = np.zeros((NoCustomers))
        for j in range(NoCustomers):
            d[j] = get_Dist(Xcenter,Ycenter,Xcord[j],Ycord[j])
            if (d[j]!=0):
                w = (1/d[j])*(1/Dem[j])
                x = x+Xcord[j]*w
                y = y+Ycord[j]*w
                W = W +w
        
        if (W!=0):
            X = x/W
            Y = y/W
        else:
            X = x
            Y=y
        New_Cost = (np.sum(np.multiply(Facil_distance(Xcord,Ycord,X,Y,NoCustomers),Dem.T)))
        improv = Cost - New_Cost
        if (improv>epsilon):
            Cost = New_Cost
            Xcenter = X
            Ycenter = Y
    
    return Xcenter,Ycenter
    

In [72]:
def Cooper_Algorithm(Xcord,Ycord,ChromoX,ChromoY,NoCustomers,NoFacil,Dem):
    
    epsilon = 10**-5
    improvement = 1
    d = np.zeros((NoCustomers,NoFacil))
    proximity = np.zeros((NoCustomers))
    for i in range(NoFacil):
        d[:,i]=Facil_distance(Xcord,Ycord,ChromoX[i],ChromoY[i],NoCustomers)
        
    Cost=(np.sum(np.multiply(d.min(axis = 1),Dem.T)))
    proximity=(d.argmin(axis=1))
    
    tmpX = np.copy(ChromoX)
    tmpY = np.copy(ChromoY)


    while (improvement>epsilon):
        
        for j in range(NoFacil):
            #print(tmpX[j],tmpY[j])
            tmpX[j],tmpY[j] = Weiszfeld(Xcord[proximity==j],Ycord[proximity==j],tmpX[j],tmpY[j],sum(proximity==j),Dem[proximity==j])
            #print(tmpX[j],tmpY[j])
        
        d = np.zeros((NoCustomers,NoFacil))
        proximity = np.zeros((NoCustomers))
        for i in range(NoFacil):
            d[:,i]=Facil_distance(Xcord,Ycord,tmpX[i],tmpY[i],NoCustomers)
        
        New_Cost=(np.sum(np.multiply(d.min(axis = 1),Dem.T)))
        proximity=(d.argmin(axis=1))
        improvement = Cost - New_Cost
        Cost=New_Cost

    return Cost, tmpX,tmpY
    
    
    
    
    
    
    

In [74]:
z1,z2,z3 = Cooper_Algorithm(Xcord,Ycord,ChromoX[0],ChromoY[0],NoCustomers,NoFacil,Demand)


In [ ]:
def Cooper_Evaluation(Xcord, Ycord,Dem,ChromoX,ChromoY,NoCustomers,NoFacil,NoChromo):
    
    
    result =np.zeros((NoChromo))
    for i in range(NoChromo):
        d = np.zeros((NoCustomers,NoFacil))
        proximity = np.zeros((NoCustomers))
        for j in range(NoFacil):
            d[:,j]=Facil_distance(Xcord,Ycord,ChromoX[i,j],ChromoY[i,j],NoCustomers)
    
        proximity=(d.argmin(axis=1))
        
        
        
        
        
        
        ### Lots missing here add
        
        
        
        
        
        
        
        
        
        
    
    
    return result